In [1]:
import pandas as pd
import numpy as np
import time
import math

In [2]:
tables_list = []
for i in range(2008, 2024, 1):
    time.sleep(0.5)
    print(i)
    if i in range(2010, 2016):
        tables_list.append(pd.read_html(f"https://en.wikipedia.org/wiki/{i}_FIFA_Ballon_d'Or"))
    elif i == 2020:
        continue
    elif i == 2016:
        tables_list.append(pd.read_html("https://en.wikipedia.org/wiki/2016_Ballon_d%27Or"))
    else:
        tables_list.append(pd.read_html(f"https://en.wikipedia.org/wiki/{i}_Ballon_d'Or"))

2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [3]:
tables_list[0][2]['year'] = [2008 for i in range(len(tables_list[0][2]))]
tables_list[0] = tables_list[0][2]

tables_list[1][2]['year'] = [2009 for i in range(len(tables_list[1][2]))]
tables_list[1] = tables_list[1][2]

tables_list[2] = pd.concat([tables_list[2][2], tables_list[2][3]], ignore_index=True)
tables_list[2]['year'] = [2010 for i in range(len(tables_list[2]))]

table = pd.concat([tables_list[3][2], tables_list[3][3]], ignore_index=True)
playas = []
for i in range(len(table)):
    if i in [0,1,2]:
        playas.append(table['Player[2]'][i])
    else:
        playas.append(table['Player[3]'][i])
table['player'] = playas
table['year'] = [2011 for i in range(len(table))]
tables_list[3] = table

n = 4
table = pd.concat([tables_list[n][2], tables_list[n][3]], ignore_index=True)
table['year'] = [2012 for i in range(len(table))]
tables_list[n] = table

n = 5
table = pd.concat([tables_list[n][2], tables_list[n][3]], ignore_index=True)
table['year'] = [2013 for i in range(len(table))]
tables_list[n] = table

n = 6
table = pd.concat([tables_list[n][2], tables_list[n][3]], ignore_index=True)
table['year'] = [2014 for i in range(len(table))]
tables_list[n] = table

n = 7
table = pd.concat([tables_list[n][2], tables_list[n][3]], ignore_index=True)
table['year'] = [2015 for i in range(len(table))]
tables_list[n] = table

tables_list[8][2]['year'] = [2016 for i in range(len(tables_list[8][2]))]
tables_list[8] = tables_list[8][2]

n=9
table = tables_list[n][2]
# table = pd.concat([tables_list[n][2], tables_list[n][3]], ignore_index=True)
table['year'] = [2017 for i in range(len(table))]
tables_list[n] = table

tables_list[10][2]['year'] = [2018 for i in range(len(tables_list[10][2]))]
tables_list[10] = tables_list[10][2]

tables_list[11] = pd.read_html("https://en.wikipedia.org/wiki/2019_Ballon_d'Or")
tables_list[11][2]['year'] = [2019 for i in range(len(tables_list[11][2]))]
tables_list[11] = tables_list[11][2]

tables_list[12][2]['year'] = [2021 for i in range(len(tables_list[12][2]))]
tables_list[12] = tables_list[12][2]

tables_list[13][2]['year'] = [2022 for i in range(len(tables_list[13][2]))]
tables_list[13] = tables_list[13][2]

# tables_list.append(pd.read_html("https://en.wikipedia.org/wiki/2023_Ballon_d'Or"))
tables_list[14][2]['year'] = [2023 for i in range(len(tables_list[14][2]))]
tables_list[14] = tables_list[14][2]

In [4]:
big_table = pd.concat(tables_list, ignore_index=True)

In [5]:
big_table.columns = [x.lower() for x in big_table.columns]

In [6]:
print(list(big_table.columns.values))

['rank', 'player', 'nationality', 'club(s)', 'points', 'year', 'percent', 'player[2]', 'player[3]', 'player', 'player[11]', 'votes', 'position', 'club']


In [7]:
# renaming columns
# big_table.columns = ['rank', 'player0', 'nationality', 'club(s)', 'points', 'year', 'percent', 'player1', 'player2', 'player3', 'player4', 'votes', 'parser0', 'parser1', 'position', 'club']
big_table.columns = ['rank', 'player0', 'nationality', 'club(s)', 'points', 'year', 'percent', 'player1', 'player2', 'player3', 'player4', 'votes', 'position', 'club']
big_table.head()

,rank,player0,nationality,club(s),points,year,percent,player1,player2,player3,player4,votes,position,club
0,1,Cristiano Ronaldo,Portugal,Manchester United,446,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Lionel Messi,Argentina,Barcelona,281,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Fernando Torres,Spain,Liverpool,179,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Iker Casillas,Spain,Real Madrid,133,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Xavi,Spain,Barcelona,97,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
new_player_list = []
count_before = big_table.shape[0]
row_remove_idx = []
for idx, row in big_table.iterrows():
    # assigning players
    p0 = row['player0']
    p1 = row['player1']
    p2 = row['player2']
    p3 = row['player3']
    p4 = row['player4']

    non_nans = []
    nan_count = 0
    for i in [p0, p1, p2, p3, p4]:
        if not isinstance(i, str) and math.isnan(i):
            nan_count += 1
        else:
            non_nans.append(i)
    
    # only one non-nan value, so we can assume correctness
    if nan_count == 4 and len(non_nans) == 1:
        new_player_list.append(non_nans[0])
    
    # if multiple non-nan values but they are the same, we can assume correctness
    elif len(set(non_nans)) == 1:
        new_player_list.append(non_nans[0])
    
    # luckily there are never multiple players in same row
    elif len(set(non_nans)) >= 1:
        print('Multiple players in same row!?')

    # no player name in row, this occurs 5 times
    elif nan_count == 5:
        # uncommenting print and looking via manual inspection shows no useful information in rows, so we can skip them
        # print(row)
        row_remove_idx.append(idx)

count_after = len(new_player_list)
print(f'Lost {count_before-count_after} rows')

big_table.drop(row_remove_idx, inplace=True)

# adding new column
big_table['player'] = new_player_list

Lost 0 rows


In [9]:
# removing old columns and crazy columns
big_table = big_table[['year', 'player', 'rank', 'points', 'votes', 'percent', 'position', 'club', 'club(s)', 'nationality']].reset_index(drop=True)
big_table

,year,player,rank,points,votes,percent,position,club,club(s),nationality
0,2008,Cristiano Ronaldo,1,446,NaN,NaN,NaN,NaN,Manchester United,Portugal
1,2008,Lionel Messi,2,281,NaN,NaN,NaN,NaN,Barcelona,Argentina
2,2008,Fernando Torres,3,179,NaN,NaN,NaN,NaN,Liverpool,Spain
3,2008,Iker Casillas,4,133,NaN,NaN,NaN,NaN,Real Madrid,Spain
4,2008,Xavi,5,97,NaN,NaN,NaN,NaN,Barcelona,Spain
...,...,...,...,...,...,...,...,...,...,...
403,2023,Jamal Musiala,26,0,NaN,NaN,Midfielder,Bayern Munich,NaN,Germany
404,2023,Nicolò Barella,27,0,NaN,NaN,Midfielder,Inter Milan,NaN,Italy
405,2023,Randal Kolo Muani,28,0,NaN,NaN,Forward,Eintracht Frankfurt,NaN,France
406,2023,Martin Ødegaard,28,0,NaN,NaN,Midfielder,Arsenal,NaN,Norway


In [10]:
# many nans in points and votes especially makes me question their usability
print(big_table.shape[0])
print(big_table['points'].isna().sum())
print(big_table['votes'].isna().sum())

408
138
385


In [11]:
# hopefully columns can just be mergeable and nans will disappear
print(big_table.shape[0])
print(big_table['club'].isna().sum())
print(big_table['club(s)'].isna().sum())

408
348
60


In [12]:
one_nan_count = 0
both_nan_count = 0
for idx, row in big_table.iterrows():
    club = row['club']
    clubs = row['club(s)']

    if club == clubs:
        print('same')

    elif isinstance(clubs, str) and not isinstance(club, str) and math.isnan(club):
        # print(f'club is nan, club(s) is {clubs}')
        one_nan_count += 1

    elif isinstance(club, str) and not isinstance(clubs, str) and math.isnan(clubs):
        # print(f'club(s) is nan, club is {club}')
        one_nan_count += 1

    elif not isinstance(club, str) and math.isnan(club) and not isinstance(clubs, str) and math.isnan(clubs):
        # print('both are nan')
        both_nan_count += 1
    
print(one_nan_count)
print(both_nan_count)

# only one_nan_counts means we can merge the columns easily

408
0


In [13]:
# removing old clubs and adding new club column with no nans
new_club = []
for idx, row in big_table.iterrows():
    club = row['club']
    clubs = row['club(s)']

    if isinstance(club, str):
        new_club.append(club)
    else:
        new_club.append(clubs)

big_table.drop(['club', 'club(s)'], axis=1, inplace=True)
big_table['club'] = new_club

In [14]:
big_table

,year,player,rank,points,votes,percent,position,nationality,club
0,2008,Cristiano Ronaldo,1,446,NaN,NaN,NaN,Portugal,Manchester United
1,2008,Lionel Messi,2,281,NaN,NaN,NaN,Argentina,Barcelona
2,2008,Fernando Torres,3,179,NaN,NaN,NaN,Spain,Liverpool
3,2008,Iker Casillas,4,133,NaN,NaN,NaN,Spain,Real Madrid
4,2008,Xavi,5,97,NaN,NaN,NaN,Spain,Barcelona
...,...,...,...,...,...,...,...,...,...
403,2023,Jamal Musiala,26,0,NaN,NaN,Midfielder,Germany,Bayern Munich
404,2023,Nicolò Barella,27,0,NaN,NaN,Midfielder,Italy,Inter Milan
405,2023,Randal Kolo Muani,28,0,NaN,NaN,Forward,France,Eintracht Frankfurt
406,2023,Martin Ødegaard,28,0,NaN,NaN,Midfielder,Norway,Arsenal


In [15]:
print(big_table.shape[0])
print(big_table['year'].isna().sum())
print(big_table['player'].isna().sum())
print(big_table['rank'].isna().sum())
print(big_table['points'].isna().sum())
print(big_table['votes'].isna().sum())
print(big_table['position'].isna().sum())
print(big_table['percent'].isna().sum())
print(big_table['nationality'].isna().sum())
print(big_table['club'].isna().sum())

408
0
0
0
138
385
348
270
150
0


In [16]:
# fill in nationalities by building dict of players and filling in after
# after googling there are actually some big players who changed nationality so it might be worth not getting rid of nans this way but getting them from another data source

nans_before = big_table['nationality'].isna().sum()

# build dict
player_nats = dict()
for idx, row in big_table.iterrows():
    player = row['player']
    nat = row['nationality']

    if player not in player_nats.keys():
        if isinstance(nat, str):
            player_nats[player] = nat

# make list corresponding to df
new_nats = []
for idx, row in big_table.iterrows():
    player = row['player']
    nat = row['nationality']

    if player in player_nats.keys():
        new_nats.append(player_nats[player])
    else:
        new_nats.append(np.nan)

# remove old column and add new
big_table.drop('nationality', axis=1, inplace=True)
big_table['nationality'] = new_nats

nans_after = big_table['nationality'].isna().sum()
print('nans before:', nans_before)
print('nans after:', nans_after)

nans before: 150
nans after: 57


In [17]:
print(big_table.shape[0])
print(big_table['year'].isna().sum())
print(big_table['player'].isna().sum())
print(big_table['rank'].isna().sum())
print(big_table['points'].isna().sum())
print(big_table['votes'].isna().sum())
print(big_table['position'].isna().sum())
print(big_table['percent'].isna().sum())
print(big_table['nationality'].isna().sum())
print(big_table['club'].isna().sum())

408
0
0
0
138
385
348
270
57
0
